<Center> Real Time Mask Detect

In [1]:
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
from imutils.video import VideoStream
import numpy as np
import imutils
import time
import cv2
import os

In [12]:
CONF = .5
def detect_and_predict_mask(frame, faceNet, maskNet):
    (h, w, _) = frame.shape
    blob = cv2.dnn.blobFromImage(frame, 1.0, (300,300), (104.0, 177.0, 123.0))
    
    faceNet.setInput(blob)
    detections = faceNet.forward()
    
    
    faces = []
    locs = []
    preds = []
    
    
    for i in range(0, detections.shape[2]):
        confidence = detections[0, 0, i, 2]
        
        if confidence >CONF:
            bbox = detections[0,0,i,3:7]* np.array([w, h, w, h])
            (startX, startY, endX, endY) = bbox.astype("int")
            
            (startX, startY) = (max(0, startX), max(0, startY))
            (endX, endY) = (min(w - 1, endX), min(h - 1, endY))
            
            face = frame[startY:endY, startX:endX]
            face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
            face = cv2.resize(face, (224, 224))
            face = img_to_array(face)
            face = preprocess_input(face)
            
            # add the face and bounding boxes to their respective
            # lists
            faces.append(face)
            locs.append((startX, startY, endX, endY))
            print("got ", confidence)
            
    # only make a predictions if at least one face was detected
    if len(faces) > 0:
        faces = np.array(faces, dtype="float32")
        preds = maskNet.predict(faces, batch_size=32)
        
#         print(faces)
#     print("locs: ", locs, "\nPreds: ", preds)
    return (locs, preds)

In [13]:
prototxtPath = r'faceDetector/deploy.prototxt'
weightsPath = r'faceDetector/res10_300x300_ssd_iter_140000.caffemodel'

faceNet = cv2.dnn.readNet(prototxtPath, weightsPath)

maskNet = load_model(r'mask_detector.model')

In [14]:
img = r'images/example_01.png'
img = cv2.imread(img)
img = imutils.resize(img, width=400)
detect_and_predict_mask(img, faceNet, maskNet)
# print(locs, preds)


got  0.9987644


([(218, 41, 268, 117)], array([[0.9950355 , 0.00496453]], dtype=float32))

In [18]:
cam = cv2.VideoCapture(0)

withMaskColor, withoutMaskColor  = (0, 255, 0), (0, 0, 255)
font = cv2.FONT_HERSHEY_SIMPLEX

while True:
    s, frame = cam.read()
    
    if not s:
        break
        
    frame = imutils.resize(frame, width=400)
    
    (locs, preds) = detect_and_predict_mask(frame, faceNet, maskNet)
    
    for (bbox, pred) in zip(locs, preds):
        
        (startX, startY, endX, endY) = bbox
        (mask, withoutMask) = pred
        
        label = "Mask" if mask> withoutMask else "No Mask"
        color = withMaskColor if label=="Mask" else withoutMaskColor

        #adding accouricy
        label = "{}: {:.2f}%".format(label, max(mask, withoutMask) * 100)

        #display label along with rectangle
        cv2.putText(frame, label, (startX, startY-10), font, .5, color, 2)
        cv2.rectangle(frame, (startX, startY), (endX, endY), color, 2)
        
        
    cv2.imshow("outPut", frame)
    key = cv2.waitKey(1) & 0xFF
	# if the `q` key was pressed, break from the loop
    if key == ord("q"):
        break
        
cam.release()        
cv2.destroyAllWindows()

got  0.9993505
got  0.998604
got  0.9988312
got  0.99916005
got  0.9986467
got  0.9980818
got  0.99855953
got  0.99930847
got  0.9995863
got  0.9998259
got  0.999821
got  0.99975795
got  0.99981934
got  0.9998455
got  0.999818
got  0.9998491
got  0.999798
got  0.999811
got  0.9997315
got  0.9995523
got  0.9996246
got  0.99967337
got  0.9995876
got  0.9996979
got  0.99973863
got  0.999795
got  0.99976283
got  0.9997223
got  0.99971634
got  0.9997009
got  0.9998703
got  0.9998797
got  0.99973625
got  0.9996531
got  0.99918085
got  0.9990502
got  0.9970368
got  0.99096864
got  0.98986316
got  0.986318
got  0.9802043
got  0.98476136
got  0.98516095
got  0.9850419
got  0.98022616
got  0.9732291
got  0.5507943
got  0.91651684
got  0.8544031
got  0.91814893
got  0.6534736
got  0.8532994
got  0.833258
got  0.8038645
got  0.8111971
got  0.81183076
got  0.61916494
got  0.87939906
got  0.94150805
got  0.9488883
got  0.99517244
got  0.99907243
got  0.9977869
got  0.99772865
got  0.57338935
got  0.